# Open SD Processing Set Demo

First extract processing set from tar file.

Tar file is uploaded to GDrive: `CASA/Temporary/uid___A002_Xc1469f_X1c33.zarr.tar.xz`

In [1]:
import os
import tarfile

ps_tar = "uid___A002_Xc1469f_X1c33.zarr.tar.xz"
ps_store = "uid___A002_Xc1469f_X1c33.ms.zarr.ps.zarr"

if not os.path.exists(ps_store):
    assert os.path.exists(ps_tar)
    with tarfile.open(ps_tar, "r:xz") as f:
        f.extractall()

## Open Processing Set with `open_processing_set`

Open processing set and filter MSv4 with scan intent (`"OBSERVE_TARGET#OFF_SOURCE"`) and spectral window name (the name containing `"FULL_RES"` to obtain the data for science spectral window).

It takes relatively long time (~1min).

In [2]:
%%time
from xradio.measurement_set import open_processing_set

intents_off = ["OBSERVE_TARGET#OFF_SOURCE"]

def is_science_spw(xdt):
    if xdt.type == "processing_set":
        return True

    if xdt.type not in ("visibility", "spectrum"):
        _ms = xdt.parent.xr_ms
    else:
        _ms = xdt.xr_ms
    try:
        partition_info = _ms.get_partition_info()
    except:
        partition_info = {}
    spectral_window_name = partition_info.get("spectral_window_name", "")
    return "FULL_RES" in spectral_window_name

ps_off = open_processing_set(
    ps_store=ps_store,
    scan_intents=intents_off
)

selected = ps_off.filter(is_science_spw)
selected

CPU times: user 57.9 s, sys: 1.25 s, total: 59.1 s
Wall time: 1min 1s


<xarray.DataTree>
Group: /
│   Attributes:
│       type:     processing_set
├── Group: /uid___A002_Xc1469f_X1c33_168
│   │   Dimensions:                     (time: 396, antenna_name: 1, frequency: 4096,
│   │                                    polarization: 2)
│   │   Coordinates:
│   │     * time                        (time) float64 3kB 1.498e+09 ... 1.498e+09
│   │     * antenna_name                (antenna_name) <U9 36B 'PM01_T704'
│   │     * frequency                   (frequency) float64 33kB 8.575e+10 ... 8.675e+10
│   │     * polarization                (polarization) <U2 16B 'XX' 'YY'
│   │       field_name                  (time) <U33 52kB dask.array<chunksize=(396,), meta=np.ndarray>
│   │       scan_name                   (time) <U21 33kB dask.array<chunksize=(396,), meta=np.ndarray>
│   │       telescope_name              (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │   Data variables:
│   │       EFFECTIVE_INTEGRATION_TIME  (time, antenna_name) float64 3kB dask.array<chunksize=(396, 1), meta=np.ndarray>
│   │       FLAG                        (time, antenna_name, frequency, polarization) bool 3MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       SPECTRUM                    (time, antenna_name, frequency, polarization) float32 13MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       TIME_CENTROID               (time, antenna_name) float64 3kB dask.array<chunksize=(396, 1), meta=np.ndarray>
│   │       VISIBILITY_CORRECTED        (time, antenna_name, frequency, polarization) complex64 26MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       WEIGHT                      (time, antenna_name, frequency, polarization) float32 13MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │   Attributes:
│   │       creation_date:     2025-10-29T06:13:44.527448+00:00
│   │       creator:           {'software_name': 'xradio', 'version': '1.0.2'}
│   │       data_groups:       {'base': {'correlated_data': 'SPECTRUM', 'date': '2025...
│   │       observation_info:  {'execution_block_UID': 'uid://A002/Xc1469f/X1c33', 'o...
│   │       processor_info:    {'sub_type': 'ALMA_ACA', 'type': 'CORRELATOR'}
│   │       schema_version:    4.0.0
│   │       type:              spectrum
│   ├── Group: /uid___A002_Xc1469f_X1c33_168/antenna_xds
│   │       Dimensions:                 (antenna_name: 1, cartesian_pos_label: 3,
│   │                                    receptor_label: 2)
│   │       Coordinates:
│   │         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│   │         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│   │           mount                   (antenna_name) <U6 24B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           polarization_type       (antenna_name, receptor_label) <U1 8B dask.array<chunksize=(1, 2), meta=np.ndarray>
│   │           station_name            (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           telescope_name          (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │       Data variables:
│   │           ANTENNA_DISH_DIAMETER   (antenna_name) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           ANTENNA_FOCUS_LENGTH    (antenna_name) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>
│   │           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
│   │       Attributes:
│   │           overall_telescope_name:  ALMA
│   │           relocatable_antennas:    True
│   │           type:                    antenna
│   ├── Group: /uid___A002_Xc1469f_X1c33_168/field_and_source_base_xds
│   │       Dimensions:                           (field_name: 1, sky_dir_label: 2,
│   │                                 

## Open Processing Set with manual filtering by accessing attributes directly

Accessing JSON attributes stored in `.zattrs` file speed up opening processing set.

In [3]:
%%time
import json
import os

import xarray as xr

contents = os.listdir(ps_store)
xdt_dict: dict[str, xr.Dataset | xr.DataTree] = {"/": xr.open_dataset(ps_store)}
for xds_name in contents:
    if xds_name.startswith("."):
        # this will not be a name of xds
        continue

    scan_name_attr = os.path.join(ps_store, xds_name, "scan_name/.zattrs")
    is_off_source = False
    if os.path.exists(scan_name_attr):
        with open(scan_name_attr, "r") as f:
            scan_name_dict = json.load(f)
        scan_intents = scan_name_dict.get("scan_intents", [])
        is_off_source = bool(set(intents_off).intersection(scan_intents))

    frequency_attr = os.path.join(ps_store, xds_name, "frequency/.zattrs")
    is_science_spw = False
    if os.path.exists(frequency_attr):
        with open(frequency_attr, "r") as f:
            frequency_dict = json.load(f)
        spectral_window_name = frequency_dict.get("spectral_window_name", "")
        is_science_spw = "FULL_RES" in spectral_window_name

    if is_off_source and is_science_spw:
        print(f"Selected {xds_name} as OFF source scan")
        xdt_dict[f"/{xds_name}"] = xr.open_datatree(
            os.path.join(ps_store, xds_name), engine="zarr", chunks={}, chunked_array_type="dask"
        )

selected2 = xr.DataTree.from_dict(xdt_dict)
selected2


Selected uid___A002_Xc1469f_X1c33_168 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_169 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_170 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_171 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_172 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_173 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_174 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_175 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_176 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_177 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_178 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_179 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_363 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_364 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_365 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_366 as OFF source scan
Selected uid___A002_Xc1469f_X1c33_367 as OFF source scan
Selected uid___A002_Xc1469f_X1c

<xarray.DataTree>
Group: /
│   Attributes:
│       type:     processing_set
├── Group: /uid___A002_Xc1469f_X1c33_168
│   │   Dimensions:                     (time: 396, antenna_name: 1, frequency: 4096,
│   │                                    polarization: 2)
│   │   Coordinates:
│   │     * time                        (time) float64 3kB 1.498e+09 ... 1.498e+09
│   │     * antenna_name                (antenna_name) <U9 36B 'PM01_T704'
│   │     * frequency                   (frequency) float64 33kB 8.575e+10 ... 8.675e+10
│   │     * polarization                (polarization) <U2 16B 'XX' 'YY'
│   │       field_name                  (time) <U33 52kB dask.array<chunksize=(396,), meta=np.ndarray>
│   │       scan_name                   (time) <U21 33kB dask.array<chunksize=(396,), meta=np.ndarray>
│   │       telescope_name              (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │   Data variables:
│   │       EFFECTIVE_INTEGRATION_TIME  (time, antenna_name) float64 3kB dask.array<chunksize=(396, 1), meta=np.ndarray>
│   │       FLAG                        (time, antenna_name, frequency, polarization) bool 3MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       SPECTRUM                    (time, antenna_name, frequency, polarization) float32 13MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       TIME_CENTROID               (time, antenna_name) float64 3kB dask.array<chunksize=(396, 1), meta=np.ndarray>
│   │       VISIBILITY_CORRECTED        (time, antenna_name, frequency, polarization) complex64 26MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │       WEIGHT                      (time, antenna_name, frequency, polarization) float32 13MB dask.array<chunksize=(396, 1, 4096, 2), meta=np.ndarray>
│   │   Attributes:
│   │       creation_date:     2025-10-29T06:13:44.527448+00:00
│   │       creator:           {'software_name': 'xradio', 'version': '1.0.2'}
│   │       data_groups:       {'base': {'correlated_data': 'SPECTRUM', 'date': '2025...
│   │       observation_info:  {'execution_block_UID': 'uid://A002/Xc1469f/X1c33', 'o...
│   │       processor_info:    {'sub_type': 'ALMA_ACA', 'type': 'CORRELATOR'}
│   │       schema_version:    4.0.0
│   │       type:              spectrum
│   ├── Group: /uid___A002_Xc1469f_X1c33_168/antenna_xds
│   │       Dimensions:                 (antenna_name: 1, cartesian_pos_label: 3,
│   │                                    receptor_label: 2)
│   │       Coordinates:
│   │         * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
│   │         * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
│   │           mount                   (antenna_name) <U6 24B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           polarization_type       (antenna_name, receptor_label) <U1 8B dask.array<chunksize=(1, 2), meta=np.ndarray>
│   │           station_name            (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           telescope_name          (antenna_name) <U4 16B dask.array<chunksize=(1,), meta=np.ndarray>
│   │       Data variables:
│   │           ANTENNA_DISH_DIAMETER   (antenna_name) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           ANTENNA_FOCUS_LENGTH    (antenna_name) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
│   │           ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>
│   │           ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
│   │       Attributes:
│   │           overall_telescope_name:  ALMA
│   │           relocatable_antennas:    True
│   │           type:                    antenna
│   ├── Group: /uid___A002_Xc1469f_X1c33_168/field_and_source_base_xds
│   │       Dimensions:                           (field_name: 1, sky_dir_label: 2,
│   │                                 